In [14]:
import torch
from autograd.engine import Value

✅ Test 1: test_sanity_check()
* Purpose: Basic sanity test — checks whether your autograd engine correctly computes forward values and backward gradients for a simple expression.

In [13]:
def test_sanity_check():

    x = Value(-4.0)
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xmg, ymg = x, y

    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xpt, ypt = x, y

    # forward pass went well
    assert ymg.data == ypt.data.item()
    # backward pass went well
    assert xmg.grad == xpt.grad.item()

In [12]:
# If both assertions pass → your engine matches PyTorch.
test_sanity_check()

#### 🧮 Test 2: test_more_ops()
* Purpose: Complex expression test — ensures your implementation handles mixed operations, chaining, broadcasting, and nonlinearities (ReLU).
* This test runs a longer and more complicated sequence of operations, involving:
>Addition, multiplication, power (**), ReLU, and division.

In [10]:
def test_more_ops():

    a = Value(-4.0)
    b = Value(2.0)
    c = a + b
    d = a * b + b**3
    c += c + 1
    c += 1 + c + (-a)
    d += d * 2 + (b + a).relu()
    d += 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g += 10.0 / f
    g.backward()
    amg, bmg, gmg = a, b, g

    a = torch.Tensor([-4.0]).double()
    b = torch.Tensor([2.0]).double()
    a.requires_grad = True
    b.requires_grad = True
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + (b + a).relu()
    d = d + 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 / f
    g.backward()
    apt, bpt, gpt = a, b, g

    tol = 1e-6
    # forward pass went well
    assert abs(gmg.data - gpt.data.item()) < tol
    # backward pass went well
    assert abs(amg.grad - apt.grad.item()) < tol
    assert abs(bmg.grad - bpt.grad.item()) < tol

In [ ]:
# If all assertions hold → your engine produces results numerically identical to PyTorch.
test_more_ops()

In [8]:
a = Value(-4.0)
b = Value(2.0)
c = a + b
d = a * b + b**3
c += c + 1
c += 1 + c + (-a)
d += d * 2 + (b + a).relu()
d += 3 * d + (b - a).relu()
e = c - d
f = e**2
g = f / 2.0
g += 10.0 / f
print(f'{g.data:.4f}') # prints 24.7041, the outcome of this forward pass
g.backward()
print(f'{a.grad:.4f}') # prints 138.8338, i.e. the numerical value of dg/da
print(f'{b.grad:.4f}') # prints 645.5773, i.e. the numerical value of dg/db

24.7041
138.8338
645.5773


In [7]:
print("🧩 BASIC TESTS")
x = Value(2)
y = Value(3)
z = Value(4)

print("x =", x)
print("y =", y)
print("z =", z)

print("\n🔹 Addition:")
print("x + y =", x + y)
print("x + 2 =", x + 2)
print("2 + x =", 2 + x)

print("\n🔹 Subtraction:")
print("y - x =", y - x)
print("5 - x =", 5 - x)
print("x - 1 =", x - 1)

print("\n🔹 Multiplication:")
print("x * y =", x * y)
print("x * 2 =", x * 2)
print("2 * x =", 2 * x)

print("\n🔹 Power:")
print("x ** 2 =", x ** 2)
print("2 ** x =", 2 ** x)

print("\n🔹 Division:")
print("y / x =", y / x)
print("x / 2 =", x / 2)
print("2 / x =", 2 / x)

# Test composite expressions
print("\n🧩 COMPLEX EXPRESSIONS")
expr1 = (x + y) * z
expr2 = (x - 1) / (y + 1)
expr3 = (x * y) + (z / x)
# This will give ZeroDivisionError: division by zero(i code it in call value)  
# expr4 = (2 + x) ** 2 / (3 - y)  

print("expr1 =", expr1)
print("expr2 =", expr2)
print("expr3 =", expr3)
# print("expr4 =", expr4)

🧩 BASIC TESTS
x = Value(data=2.0)
y = Value(data=3.0)
z = Value(data=4.0)

🔹 Addition:
x + y = Value(data=5.0)
x + 2 = Value(data=4.0)
2 + x = Value(data=4.0)

🔹 Subtraction:
y - x = Value(data=1.0)
5 - x = Value(data=3.0)
x - 1 = Value(data=1.0)

🔹 Multiplication:
x * y = Value(data=6.0)
x * 2 = Value(data=4.0)
2 * x = Value(data=4.0)

🔹 Power:
x ** 2 = Value(data=4.0)
2 ** x = Value(data=4.0)

🔹 Division:
y / x = Value(data=1.5)
x / 2 = Value(data=1.0)
2 / x = Value(data=1.0)

🧩 COMPLEX EXPRESSIONS
expr1 = Value(data=20.0)
expr2 = Value(data=0.25)
expr3 = Value(data=8.0)
